## Table 5 (Journal of Climate submission; Molina et al.)

Table 5. Mean thermocline depth in meters, represented as the depth of the 20◦C isotherm (Kessler 1990),across the tropical Pacific  during El Niño, La Niña, and mean climatology. ENSO events and climatology are derived from the years 800-1599 for the CESM1 control, 201-500 for Global and Pacific experiments, and 101-250 for the Pacific Salt experiment.

**Table by: Maria J. Molina**

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import cftime

In [2]:
from config import directory_figs, directory_data

In [3]:
#def pop_lon_indx():
#    """
#    Extract mask for the pacific slab region. Mask contains ones and nans. (Previous version for just nino3.4)
#    """
#    for_lon = xr.open_dataset('/glade/scratch/molina/amoc_exp/b.e11.B1850LENS.f09_g16.FWAtSalG02Sv.pop.h.SST.000101-005012.nc')
#    mask = for_lon['SST'].where((for_lon['TLAT']<5) & (for_lon['TLAT']>-5) & (for_lon['TLONG']>-170+360) & (for_lon['TLONG']<-120+360), 
#                                 drop=False).isel(z_t=0, time=0).values
#    return np.where(np.isnan(mask), np.nan, 1)

def pop_lon_indx():
    """
    Extract mask for the pacslab region. Mask contains ones and nans.
    """
    for_lon = xr.open_dataset(f'{directory_data}b.e11.B1850LENS.f09_g16.FWAtSalG02Sv.pop.h.SST.000101-005012.nc')
    mask = for_lon['SST'].where((for_lon['TLAT']<10) & (for_lon['TLAT']>-10) & (for_lon['TLONG']>160) & (for_lon['TLONG']<-80+360), 
                                 drop=False).isel(z_t=0, time=0).values
    return np.where(np.isnan(mask), np.nan, 1)

def compute_iso(threedim_array, mask):
    """
    Create array of depth of isotherm using 3d iso array and 2d mask.
    Args:
        threedim_array (numpy array): Isotherm values.
        mask (numpy array): Mask from pop_lon_indx.
    Returns:
        One dimensional array across Pacific slab region.
    """
    newmask = np.nanmean(np.nanmean(threedim_array[:,:,:] * mask[np.newaxis,:,:], axis=0), axis=0) * 0.01
    return newmask[~np.isnan(newmask)]

def for_time_series(threedim_array, mask):
    """
    Create array of depth of isotherm using 3d iso array and 2d mask.
    Args:
        threedim_array (numpy array): Isotherm values.
        mask (numpy array): Mask from pop_lon_indx.
    Returns:
        One dimensional array across Pacific slab region.
    """
    newmask = np.nanmean(threedim_array[:,:,:] * mask[np.newaxis,:,:], axis=1) * 0.01
    return newmask[~np.isnan(newmask)]

In [4]:
# grab lon indxs
lon_array_locs = pop_lon_indx()

In [9]:
# slab isotherms
iso20_g02sv = xr.open_dataset(
    f'{directory_data}iso20c_FWAtSalG02Sv.nc').sel(
    TIME=slice(cftime.DatetimeNoLeap(201, 1, 1, 0, 0),cftime.DatetimeNoLeap(500, 12, 1, 0, 0)))['DEPTH_OF_20C'].resample(TIME='QS-DEC').mean(skipna=True)

iso20_g04sv = xr.open_dataset(
    f'{directory_data}iso20c_FWAtSalG04Sv.nc').sel(
    TIME=slice(cftime.DatetimeNoLeap(201, 1, 1, 0, 0),cftime.DatetimeNoLeap(500, 12, 1, 0, 0)))['DEPTH_OF_20C'].resample(TIME='QS-DEC').mean(skipna=True)

iso20_p02sv = xr.open_dataset(
    f'{directory_data}iso20c_FWAtSalP02Sv.nc').sel(
    TIME=slice(cftime.DatetimeNoLeap(201, 1, 1, 0, 0),cftime.DatetimeNoLeap(500, 12, 1, 0, 0)))['DEPTH_OF_20C'].resample(TIME='QS-DEC').mean(skipna=True)

iso20_p04sv = xr.open_dataset(
    f'{directory_data}iso20c_FWAtSalP04Sv.nc').sel(
    TIME=slice(cftime.DatetimeNoLeap(201, 1, 1, 0, 0),cftime.DatetimeNoLeap(500, 12, 1, 0, 0)))['DEPTH_OF_20C'].resample(TIME='QS-DEC').mean(skipna=True)

iso20_psalt = xr.open_dataset(
    f'{directory_data}iso20c_FWPaSalP04Sv.nc').sel(
    TIME=slice(cftime.DatetimeNoLeap(101, 1, 1, 0, 0),cftime.DatetimeNoLeap(250, 12, 1, 0, 0)))['DEPTH_OF_20C'].resample(TIME='QS-DEC').mean(skipna=True)

iso20_cntrl = xr.open_dataset(
    f'{directory_data}iso20c_005.nc').sel(
    TIME=slice(cftime.DatetimeNoLeap(800, 1, 1, 0, 0),cftime.DatetimeNoLeap(1599, 12, 1, 0, 0)))['DEPTH_OF_20C'].resample(TIME='QS-DEC').mean(skipna=True)


In [10]:
iso20_g02sv = iso20_g02sv[iso20_g02sv['TIME.month']==12].values
iso20_g04sv = iso20_g04sv[iso20_g04sv['TIME.month']==12].values
iso20_p02sv = iso20_p02sv[iso20_p02sv['TIME.month']==12].values
iso20_p04sv = iso20_p04sv[iso20_p04sv['TIME.month']==12].values
iso20_psalt = iso20_psalt[iso20_psalt['TIME.month']==12].values
iso20_cntrl = iso20_cntrl[iso20_cntrl['TIME.month']==12].values

In [17]:
iso20_g02sv = compute_iso(iso20_g02sv, lon_array_locs)
iso20_g04sv = compute_iso(iso20_g04sv, lon_array_locs)
iso20_p02sv = compute_iso(iso20_p02sv, lon_array_locs)
iso20_p04sv = compute_iso(iso20_p04sv, lon_array_locs)
iso20_psalt = compute_iso(iso20_psalt, lon_array_locs)
iso20_cntrl = compute_iso(iso20_cntrl, lon_array_locs)

/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: Mean of empty slice


In [18]:
nino_iso20 = xr.open_dataset(f'{directory_data}ninoslabs_DEPTH_OF_20C.nc')
nina_iso20 = xr.open_dataset(f'{directory_data}ninaslabs_DEPTH_OF_20C.nc')

In [19]:
print(np.around(iso20_cntrl.mean(),1))
print(np.around(iso20_g02sv.mean(),1))
print(np.around(iso20_g04sv.mean(),1))
print(np.around(iso20_p02sv.mean(),1))
print(np.around(iso20_p04sv.mean(),1))
print(np.around(iso20_psalt.mean(),1))

123.6
119.4
124.7
121.6
123.6
124.7


In [20]:
print(np.around(nino_iso20['cntrl_nino'].mean().values,1))
print(np.around(nino_iso20['g02sv_nino'].mean().values,1))
print(np.around(nino_iso20['g04sv_nino'].mean().values,1))
print(np.around(nino_iso20['p02sv_nino'].mean().values,1))
print(np.around(nino_iso20['p04sv_nino'].mean().values,1))
print(np.around(nino_iso20['psalt_nino'].mean().values,1))

124.0
118.0
123.4
121.9
123.3
124.3


In [21]:
print(np.around(nina_iso20['cntrl_nina'].mean().values,1))
print(np.around(nina_iso20['g02sv_nina'].mean().values,1))
print(np.around(nina_iso20['g04sv_nina'].mean().values,1))
print(np.around(nina_iso20['p02sv_nina'].mean().values,1))
print(np.around(nina_iso20['p04sv_nina'].mean().values,1))
print(np.around(nina_iso20['psalt_nina'].mean().values,1))

123.5
120.1
125.3
121.5
123.7
124.9
